In [1]:
import re
import time

In [2]:
# <i class="icon icon-right"></i>
# class="icon icon-right"
# <li id="bPageNextPage" class="bPageControlButton pagination-item" style="display: none;">
def nextPage(browser,text):
    '''
    页面跳转到下一页
    '''
    pat = r'<li id="bPageNextPage" class="bPageControlButton pagination-item" style="display: none;">'
    if re.search(pat,text) == None:
#         print(1111)
        browser.find_element_by_id('bPageNextPage').click()  ## brower变成了下一页
        time.sleep(0.5)
        return 1
#         return 0  ## 无法下一页
    else:
#         browser.find_element_by_id('bPageNextPage').click()  ## brower变成了下一页
#         time.sleep(0.5)
        return 0

In [3]:
# # url = 'http://www.gdzwfw.gov.cn/portal/affairs-public-detail?qzqdCode=00288efa-bc84-4176-993f-8e08da4e56f0&amp;deptCode=MB2C94128'
# url = 'http://www.gdzwfw.gov.cn/portal/affairs-public-detail?qzqdCode=00354c85-3de2-45bd-aa91-ea6f6d0350ba&amp;deptCode=007542689'
# from selenium import webdriver
# bro = webdriver.Firefox(log_path = '33.1og')
# bro.get(url)
# time.sleep(4)
# text = bro.page_source

# nextPage(bro,text)

In [4]:
def replace_text(text):
    text = text.replace('\n','').replace('\t','').replace(' ','')
    return text

In [5]:
def replace_brac(text):
    ''' 去除掉<p...></p> <a ...> 等内容
    而如果其中href = 后面的字符串中包含网址是特定网址
    则保留该网址'''
    prefix = 'http://www.gdzwfw.gov.cn'
    urls = re.findall('href="(.*?)"',text)
#     print(urls)
    if len(urls) == 0:
        url = ''
    else:
        url = urls[0]
        if 'gov.cn' in url:
            if url.startswith('http'):
                pass
#                 url = url
            else: ## 图片url = //static.gdzwfw.gov.cn/guide/files/21...
                url = prefix + url
                
        elif url.startswith('http'): ## 点击访问
            url = url
        else:  ## href = javascript()
            url = ''
    text = re.sub('<.*?>','',text)
    if url == '':
        return text
    else:
        return text+"("+url+")"

In [6]:
def find_urls(text):
    pat = '<a class="table-item-title" href="(.*?)".*?>.*?</a></div></td>'
    url_list = re.findall(pat,text)
    return url_list

In [14]:
def S_search(pat,text):
    t = re.search(pat,text,re.S)
    if t == None:
        text1 = ''
    else:
        text1 = t.groups()[0]
    return text1

In [ ]:
## pat1 = r'<th>(.*?)</th>.*?<td>(.*?)</td>'

In [1]:
def handle_time(v,btm):
    '''
    input: 法定**时限的网页源码
    out: 格式化后的信息  [ 说明，单位，数值 ]
    '''
    rer = ['None','None','None']
    res = []
    text1 = v
    if 'javascrip' in text1 and 'data-target' in text1:
        patj = 'data-target="(.*?)"'
        idsx = S_search(patj,text1)
        if idsx == "":
            return rer
        else:
            idsx = idsx.replace('#','')
            text2 = btm.find_all(id = idsx)
            if len(text2) != 0:
                text2 = str(text2[0])
            else:
                text2 = ''
            patbd = '<div class="modal-bd">.*?<p>(.*?)</p>'
            text3 = S_search(patbd,text2)
            res.append(replace_text(text3))
        v = replace_brac(v)
        v = replace_text(v)
        v = re.split('[()]',v)
        if len(v) < 2:
            res.append('无')
            res.append('无')
        else:
            res.append(v[1])
            res.append(v[0])
        return res
    else:
        return rer